###     Extração/tratamento/carga de dados da API do SALIC

    Fonte: http://api.salic.cultura.gov.br/doc/

    Faz a leitura de projetos da lei de incentivo à cultura (antiga Lei Rouanet) entre 2010 e 2022

    Desenvolvido na equipe descentralizada de dados (DADOS/CGEBC/DS/SFC) da Controladoria-Geral da União (CGU):

    Rodrigo Moreira da Silva e Victorio Takahashi Chu 

In [1]:
import pandas as pd
import numpy as np
import sqlalchemy
import urllib
import requests
import json
import time
import glob  
import os
from tqdm import tqdm 
from datetime import datetime

### Parte 1 - Extração
#### Lendo e gravando dados de PRONAC da API do SALIC

In [8]:
#Data da extração
data = datetime.today().strftime('%Y-%m-%d')

#Pasta para gravar os dados
os.makedirs(f'./dados_api_{data}',exist_ok=True)
pasta = (f'dados_api_{data}') + os.sep 

In [62]:
def le_pronac(pronac, data_atual, pasta_dados):

    headers = {'accept': 'application/json',}
    params = {'format': 'json',}

    pronacs_api = ''
    data = data_atual
    pasta = pasta_dados 

    try:
        r = requests.get(f'http://api.salic.cultura.gov.br/v1/projetos/{pronac}', params=params, headers=headers)
        if r.status_code == 200:
            status_api = 'Operação bem-sucedida'
            with open(f'{pasta}pronac_{pronac}_{data}.json', 'w') as f:
                json.dump(r.text, f)
        elif r.status_code == 404:
            status_api = 'Projeto com PRONAC fornecido não encontrado'
        elif r.status_code == 405:
            status_api = 'PRONAC inválido' 
        elif r.status_code == 503:
            status_api = 'Erro interno'
        else:
            status_api = 'Desconhecido'
    except:
        status_api = 'Erro de conexão'

    return status_api

In [63]:
def chama_leitura_pronacs(lista, data, pasta):
    contador = 1
    dic = {}
    for pronac in tqdm(lista):
        dic[pronac] = le_pronac(pronac, data, pasta)
        contador += 1
        if contador == 100:
            contador = 1
            time.sleep(5)
    return dic

In [ ]:
#Anos que serão pesquisados: 2010 a 2022
#O PRONAC é formado por 6 dígitos: AASSSS onde AA é o ano do projeto e SSSS é um sequencial que vai de 0001 a 9999
#Entre os anos de 2010 e 2015, houve mais de 10.000 projetos. Então o sequencial deve ser mudado para 5 digitos

#Projetos de 2010 a 2022
pronacs = list(range(100000,230000))
#complemento dos pronacs
pronacs_2010 = list(range(1010000,1015000))
pronacs_2011 = list(range(1110000,1115000))
pronacs_2012 = list(range(1210000,1215000))
pronacs_2013 = list(range(1310000,1315000))
pronacs_2014 = list(range(1410000,1415000))
pronacs_2015 = list(range(1510000,1515000))
#Junção de todos sos pronacs
pronac_todos = pronacs + pronacs_2010 + pronacs_2011 + pronacs_2012 + pronacs_2013 + pronacs_2014 + pronacs_2015

In [ ]:
api_status_pronac = {}
api_status_pronac = chama_leitura_pronacs(pronac_todos, data, pasta)

In [ ]:
api_status_pronac_df = pd.DataFrame(list(api_status_pronac.items()), columns=['pronac','status'])
api_status_pronac_df['status'].value_counts()

In [ ]:
tentativa_leitura = 0

#Separa os Pronac que tiveram erro na leitura e tenta ler novamente (até 8 vezes)
while tentativa_leitura < 8:

    #Verifica quantos pronacs tiveram erro de leitura
    pronac_problema = list(api_status_pronac_df[(api_status_pronac_df['status'] == 'Erro de conexão') | 
                                                (api_status_pronac_df['status'] == 'Erro interno') | 
                                                (api_status_pronac_df['status'] == 'Desconhecido')]['pronac'])

    #Se for zero problema, sai do loop
    if len(pronac_problema) == 0:
        break

    #Lê os pronacs que tiveram problema na rodada anterior
    aux_dic = chama_leitura_pronacs(pronac_problema, data, pasta)
    #Transforma o dicionário em dataframe da mesma forma que foi feito para o api_status_pronac_df
    aux_df = pd.DataFrame(list(aux_dic.items()), columns=['pronac','status'])
    
    #Concatena os dataframes api_status_pronac_df e aux_df
    #Remove os pronacs duplicados mantendo o status mais recente
    api_status_pronac_df = pd.concat([api_status_pronac_df, aux_df])
    api_status_pronac_df.drop_duplicates(subset=['pronac'], keep='last', inplace=True)
    api_status_pronac_df.reset_index(drop=True, inplace=True)

    tentativa_leitura += 1

In [ ]:
#Grava em arquivos os pronacs que ainda apresentaram problema de leitura após 5 tentativas
if tentativa_leitura >=5:
    print(f'Após 5 tentativas de leitura, os pronacs que ainda apresentam problema estão no arquivo pronacs_problema.csv')
    pronacs_problema = (api_status_pronac_df[(api_status_pronac_df['status'] == 'Erro de conexão') | 
                                             (api_status_pronac_df['status'] == 'Erro interno') | 
                                             (api_status_pronac_df['status'] == 'Desconhecido')]).copy()
    
    arquivo = f'pronacs_problema_{data}.csv'
    pronacs_problema.to_csv(arquivo)


In [ ]:
arquivo = f'status_pronac_{data}.csv'
api_status_pronac_df.to_csv(arquivo)

In [ ]:
api_status_pronac_df['status'].value_counts()

### Parte 2 - Tratamento
#### Lendo dados gravados do SALIC (json para dataframe)

In [ ]:
#Cria o dataframe vazio e depois preenche com todos os dados lidos pela API
pronacs_df = pd.DataFrame(columns=[ 'etapa', 'providencia', 'area', 'enquadramento', 'objetivos',
                                    'ficha_tecnica', 'situacao', 'outras_fontes', 'acessibilidade',
                                    'sinopse', 'nome', 'cgccpf', 'mecanismo', '_links', 'segmento',
                                    'PRONAC', 'estrategia_execucao', 'valor_aprovado', 'justificativa',
                                    'resumo', 'valor_solicitado', 'especificacao_tecnica', '_embedded',
                                    'municipio', 'data_termino', 'UF', 'impacto_ambiental',
                                    'democratizacao', 'valor_projeto', 'proponente', 'ano_projeto',
                                    'data_inicio', 'valor_captado', 'valor_proposta'])

#Lê todos os arquivos json que foram extraídos da api do SALIC
files = glob.glob(f'{pasta}*.json')
for fname in files:
    with open(fname, 'r') as f:
        data = f.read()
        json_data = json.loads(data)
        df = pd.read_json(json_data, lines=True)
        pronacs_df = pd.concat([pronacs_df, df], ignore_index=True)

In [ ]:
pronacs_df.rename(columns={'nome':'nome_projeto'}, inplace=True)
pronacs_df.rename(columns={'proponente':'nome'}, inplace=True)

In [ ]:
pronacs_df.columns

In [ ]:
pronacs_df.rename(columns={'nome':'nome_proponente'}, inplace=True)

In [ ]:
pronacs_df_simplificado = pronacs_df[['PRONAC','area', 'enquadramento', 'situacao', 'nome_projeto', 'cgccpf', 'nome_proponente','mecanismo', 
                                      'segmento', 'resumo', 'valor_solicitado', 'municipio', 'UF', 'ano_projeto', 'data_inicio', 'data_termino',
                                      'valor_projeto', 'valor_aprovado', 'valor_captado', 'valor_proposta','outras_fontes',]].copy()

In [ ]:
#Criando dataframe com os documentos de pagamentos
df_relacao_pagamentos = pd.DataFrame()

for i in range(len(pronacs_df['_embedded'])):
    PRONAC =  pronacs_df['PRONAC'][i]
    df = pd.DataFrame(pronacs_df['_embedded'][i]['relacao_pagamentos'], \
                      columns = ['id_planilha_aprovacao',
                                'justificativa',
                                'data_pagamento',
                                'nome',
                                'cgccpf',
                                'data_aprovacao',
                                'valor_pagamento',
                                'nm_arquivo',
                                'id_arquivo',
                                'nome_fornecedor',
                                'id_comprovante_pagamento',
                                'nr_documento_pagamento',
                                'tipo_documento'])
    df['PRONAC'] = PRONAC
    df_relacao_pagamentos = pd.concat([df_relacao_pagamentos,df], ignore_index=True)
df_relacao_pagamentos.head()

In [ ]:
#PRONACs que possuem algum tipo de documento de pagamento
df_relacao_pagamentos['PRONAC'].value_counts()

In [ ]:
df_relacao_pagamentos_agrupado = df_relacao_pagamentos.groupby(['PRONAC', 'cgccpf', 'nome_fornecedor'])['valor_pagamento'].sum().to_frame()
df_relacao_pagamentos_agrupado = df_relacao_pagamentos_agrupado.reset_index()
df_relacao_pagamentos_agrupado.rename(columns={'nome_fornecedor':'nome'}, inplace=True)
df_relacao_pagamentos_agrupado

In [ ]:
df_relacao_pagamentos_agrupado

In [ ]:
# Dataframe com informações de captações 
df_captacao = pd.DataFrame()
for i in range(len(pronacs_df['_embedded'])):
    df = pd.DataFrame(pronacs_df['_embedded'][i]['captacoes'], \
                      columns = ['PRONAC','valor','data_recibo','nome_projeto','cgccpf','nome_doador'])
    df_captacao = pd.concat([df_captacao,df], ignore_index=True)
df_captacao
df_captacao.head()

In [ ]:
#PRONACs que tiveram captação de recursos
df_captacao['PRONAC'].value_counts()

In [ ]:
df_captacao_agrupado = df_captacao.groupby(['PRONAC', 'cgccpf', 'nome_doador'])['valor'].sum().to_frame()
df_captacao_agrupado = df_captacao_agrupado.reset_index()
df_captacao_agrupado.rename(columns={'nome_doador':'nome'}, inplace=True)
df_captacao_agrupado

### Parte 3 - Carga
#### Gravando dados em csv

In [ ]:
pronacs_df_simplificado.to_csv('SALIC.csv', index=False, encoding='utf-8')
df_relacao_pagamentos.to_csv('SALIC_rel_pagamentos.csv', index=False, encoding='utf-8')
df_captacao.to_csv('SALIC_captacao.csv',  index=False, encoding='utf-8')